In [ ]:
# set the number of threads for many common libraries
from os import environ
N_THREADS = '1'
environ['OMP_NUM_THREADS'] = N_THREADS
environ['OPENBLAS_NUM_THREADS'] = N_THREADS
environ['MKL_NUM_THREADS'] = N_THREADS
environ['VECLIB_MAXIMUM_THREADS'] = N_THREADS
environ['NUMEXPR_NUM_THREADS'] = N_THREADS
# https://superfastpython.com/numpy-number-blas-threads/

In [ ]:
import pandas as pd
import numpy as np
from itertools import chain

# Hi-C utilities imports:
import cooler
import bioframe
import cooltools
from cooltools.lib.numutils import fill_diag

# Visualization imports:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
from matplotlib.gridspec import GridSpec

from itertools import cycle

# from ipywidgets import interact, fixed
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
import cooltools.lib.plotting
# just to get to the fall colormap ...

### Import modified "guts" of the dotfinder submodule from `helper_func` file

In [ ]:
from helper_func import (
    draw_kernel,
    get_adjusted_expected_tile_OE,
    score_tile_custom_cols_OE,
    score_pixels_only_OE,
    get_adjusted_expected_tile_more_columns,
    score_tile_custom_cols,
    score_pixels_only,
    rectangles_around_dots,
)
score_tile_custom_cols_OE
# turns out still need some of the dotfinder guts in here
from cooltools.api.dotfinder import bp_to_bins, generate_tiles_diag_band
from cooltools.lib.numutils import LazyToeplitz
from cooltools.lib.common import assign_regions

# from datashader.mpl_ext import dsshow, alpha_colormap
# import datashader as ds
# import datashader.transfer_functions as tf
from functools import partial
from data_catalog import bws, bws_vlim, telo_dict

# optics ...
from sklearn.cluster import OPTICS
# find peaks to define anchors ...
from scipy.signal import find_peaks

### pick a dataset and binsize to work on ...

In [ ]:
# # ! pip install --upgrade --no-cache --no-deps --ignore-install cooler
# # ls /home/dekkerlab/dots-test
# # import higlass as hg
# import jscatter
import scipy
import logging
import multiprocess as mp
# import mpire for nested multi-processing
from mpire import WorkerPool

In [ ]:
# 10 kb is a resolution at which one can clearly see "dots":
binsize = 10_000
# cooler files that we'll work on :
telo_clrs = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize}") for _k, _path in telo_dict.items() }

### pick a region to work on ...

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms_full = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
# only autosomal chromosomes ...
included_arms = hg38_arms_full["name"].to_list()[:44]
hg38_arms = hg38_arms_full[hg38_arms_full["name"].isin(included_arms)].reset_index(drop=True)

### pre-calculate expected for the cooler ...

In [ ]:
def _job(packed_data, sample):
    # packed data -> exp_kwargs and a dict with coolers for each sample
    exp_kwargs, clr_dict = packed_data
    _clr = clr_dict[sample]
    # in order to use spawn/forkserver we have to import for worker
    from cooltools import expected_cis
    _exp = expected_cis( _clr, **exp_kwargs)
    return (sample, _exp)

# define expected parameters in the form of kwargs-dict:
exp_kwargs = dict(
    view_df=hg38_arms,
    intra_only=False,
    nproc=12
)

# have to use daemon=False, because _job is multiprocessing-based already ...
with WorkerPool(
    n_jobs=8,
    daemon=False,
    shared_objects=( exp_kwargs, telo_clrs ),
    start_method="spawn",  # little faster than spawn, fork is the fastest
    use_dill=True,
) as wpool:
    results = wpool.map(_job, telo_clrs, progress_bar=True)

# sort out the results ...
telo_exps_cis = {sample: _exp for sample, _exp in results}
# # old way of doing it
# telo_exps_cis = {k: cooltools.expected_cis( _clr, **exp_kwargs) for k, _clr in telo_clrs.items()}

## generate custom kernels - similar to original from hiccups

In [ ]:
# define stripy kernels for small compartment detection ...
def get_stripy_kernels_new(halfwidth):
    """
    halfwidth : int
        half width of the kernel, kernel size must be odd number in both dimensions

    returns :
    dictionaty with kernels
    """
    # kernel width defined - odd dimensions ...
    kwidth = (2*halfwidth + 1)
    # define stripe width
    stripe_width = kwidth // 3

    # create a grid of coordinates from -h to +h, to define round kernels
    x, y = np.meshgrid(
        np.linspace(-halfwidth, halfwidth, kwidth),
        np.linspace(-halfwidth, halfwidth, kwidth),
    )

    # define horizontal and vertical stripes
    maskv = ((x < stripe_width - halfwidth) | (x > halfwidth - stripe_width))
    maskv = maskv & ((y >= stripe_width - halfwidth) & (y <= halfwidth - stripe_width))
    maskvmid = ~maskv & ((y >= stripe_width - halfwidth) & (y <= halfwidth - stripe_width))
    maskh = ((y < stripe_width - halfwidth) | (y > halfwidth - stripe_width))
    maskh = maskh & ((x >= stripe_width - halfwidth) & (x <= halfwidth - stripe_width))
    maskhmid = ~maskh & ((x >= stripe_width - halfwidth) & (x <= halfwidth - stripe_width))

    # new kernels with more round donut and lowleft masks:
    return {
        f'mid': maskvmid,
        f'v{halfwidth}': maskv,
        f'h{halfwidth}': maskh,
    }

In [ ]:
# define stripy kernels of different sizes
k4 = get_stripy_kernels_new(halfwidth=3)
k7 = get_stripy_kernels_new(halfwidth=7)  # this is the one we'd need @10kb ...
kl = get_stripy_kernels_new(halfwidth=10)


# plot rounded kernels
fig, axs = plt.subplots(ncols=len(k4), nrows=len([k4, k7, kl]), figsize=(len(k4)*2.5, len([k4, k7, kl])*2.5), squeeze=False)
for ax_row, ks in zip(axs, [k4, k7, kl]):
    for ax, (ktype, kernel) in zip(ax_row, ks.items()):
        imk = draw_kernel(kernel, ax, kernel_name=ktype,cmap="plasma")

# Work on a particular clr/exp pair: mostly the dRGap 5hr sample, revisit MEGA dRGap as well ...

In [ ]:
clr = telo_clrs["p5hR1R2"]
exp = telo_exps_cis["p5hR1R2"]
# got clr, exp and hg38_arms ...
# create a special version of exp - indexed by region1,2  and dist:
exp_indexed = exp.set_index(["region1", "region2", "dist"]).sort_index()

### Define all parameters needed for enriched pixels generation

In [ ]:
# binsize - defined
max_loci_separation = 30_000_000  # max genomic separation we are going to consider
tile_size=10_000_000  # technical paramter for chunking

kernel_width = max(len(k) for k in k7.values())  # 2*w+1
kernel_half_width = int((kernel_width - 1) / 2)  # former w parameter
max_nans_tolerated=11
nproc=12

# define square tiles (chunks of the heatmap) to run convolutions on ...
tiles = list(
    generate_tiles_diag_band(
        clr,
        hg38_arms,
        kernel_half_width,
        bp_to_bins(tile_size, binsize),  # bp -> bins
        bp_to_bins(max_loci_separation, binsize),  # bp -> bins
    )
)

### some interactive exploration of the heatmap tiles ...

In [ ]:
w = widgets.IntSlider(value=24, min=0, max=100)

@interact(i=w)
def f(i):
    # selected tile is:
    the_tile = tiles[i]
    print("selected the tile near the region of interest ...")
    print(the_tile)
    # display that region of interest as a heatmap ...

    # region_name, tile_span_i, tile_span_j = 'chr1_p', (2995, 3505), (2995, 3505)
    region_name, tile_span_i, tile_span_j = the_tile
    tile_start_ij = (tile_span_i[0], tile_span_j[0])
    lazy_exp = LazyToeplitz(
        exp_indexed.loc[region_name, region_name]["balanced.avg"].to_numpy()
    )
    # RAW observed matrix slice:
    observed = clr.matrix()[slice(*tile_span_i), slice(*tile_span_j)]
    expected = lazy_exp[slice(*tile_span_i), slice(*tile_span_j)]

    f, (axleft, axright) = plt.subplots(nrows=1,ncols=2,figsize=(24,21),sharex=True,sharey=True)

    axleft.imshow(
        observed,
        cmap="fall",
        interpolation="none",
        norm=LogNorm(0.0001, 0.01)
    )
    axright.imshow(
        (observed/expected),
        cmap="RdBu_r",
        interpolation="none",
        norm=LogNorm(0.25, 4)
    )

### Apply some filters for fun - gaussian "smoothing", median filter etc

In [ ]:
w = widgets.IntSlider(value=24, min=0, max=100)

@interact(i=w)
def f(i):
    # selected tile is:
    the_tile = tiles[i]
    print("selected the tile near the region of interest ...")
    print(the_tile)
    # display that region of interest as a heatmap ...

    # region_name, tile_span_i, tile_span_j = 'chr1_p', (2995, 3505), (2995, 3505)
    region_name, tile_span_i, tile_span_j = the_tile
    tile_start_ij = (tile_span_i[0], tile_span_j[0])
    lazy_exp = LazyToeplitz(
        exp_indexed.loc[region_name, region_name]["balanced.avg"].to_numpy()
    )
    # RAW observed matrix slice:
    observed = clr.matrix()[slice(*tile_span_i), slice(*tile_span_j)]
    expected = lazy_exp[slice(*tile_span_i), slice(*tile_span_j)]

    f, (axleft, axright) = plt.subplots(nrows=1,ncols=2,figsize=(24,21),sharex=True,sharey=True)

    OE = (observed/expected)

    gOE = scipy.ndimage.gaussian_filter(OE, sigma=1, order=0, mode='reflect', cval=0.0, truncate=3.0)
    # gOE = scipy.ndimage.median_filter(OE, size=3, mode='constant', cval=0.0)
    # gOE = scipy.ndimage.gaussian_gradient_magnitude(OE, sigma=1, mode='constant', cval=0.0)
    # gOE = scipy.ndimage.prewitt(OE, axis=0, mode='constant', cval=0.0)
    # gOE = scipy.ndimage.gaussian_gradient_magnitude(OE, sigma=2, mode='reflect', cval=0.0)

    axleft.imshow(
        gOE,
        cmap="RdBu_r",
        interpolation="none",
        norm=LogNorm(0.25, 4)
    )
    axright.imshow(
        OE,
        cmap="RdBu_r",
        interpolation="none",
        norm=LogNorm(0.25, 4)
    )

# Run convolution on the selected tile ...
 - we use `v7`, `h7`, `mid` kernels
 - `nnz_threshold` is set to 33% of `k_sum.sum()` (tried 40% before)
 - `enrichment_threshold` is set to **2**
 - `max_nans_tolerated` is **11**
 - `max_loci_separation` is at `30_000_000`

In [ ]:
# columns to return
cols_to_return = ["bin1_id", "bin2_id", "count"] #, "expected_raw", "oe"]
cols_to_return += [f"la_exp.{k}.value" for k in k7]
cols_to_return += [f"convolution_ratio.{k}" for k in k7]
cols_to_return += [f"la_exp.{k}.zeros" for k in k7]

def extract_filtered_pixels(df, enrichment_threshold, nnz_threshold):
    # make df more lightweight - drop a few columns
    cols_to_drop = [f"la_exp.{k}.value" for k in k7]
    df = df.drop(columns=cols_to_drop, inplace=False, errors='raise')
    # first define a few extra columns etc
    df["dist"] = df["bin2_id"] - df["bin1_id"]
    df["zl"] = df[[f"la_exp.{k}.zeros" for k in ["v7","h7","mid"]]].sum(axis=1)

    # filtering queries:
    query_v = f"(`convolution_ratio.mid` >= {enrichment_threshold} * `convolution_ratio.v7`)"
    query_h = f"(`convolution_ratio.mid` >= {enrichment_threshold} * `convolution_ratio.h7`)"

    return df.query(f"(zl <= {nnz_threshold}) & ( {query_v} | {query_h} )"
    )


def score_pixels_OE_bulk(
    clr,
    expected_indexed,
    expected_value_col,
    clr_weight_name,
    tiles,
    kernels,
    max_nans_tolerated,
    loci_separation_bins,
    nproc,
    cols_to_return = None,
):
    logging.info(f"convolving {len(tiles)} tiles to build histograms for lambda-bins")

    # ########################################################
    # these are important thresholding parameters right here !
    # ########################################################
    _enrichment_threshold = 2.0
    k_sum = sum([k.astype(int) for n,k in kernels.items() if n in ["v7","h7","mid"]])
    _nnz_threshold = 0.33 * k_sum.sum()

    # to score per tile - a function of a single argument :
    _score = partial(
        score_tile_custom_cols_OE,
        clr=clr,
        expected_indexed=expected_indexed,
        expected_value_col=expected_value_col,
        clr_weight_name=clr_weight_name,
        kernels=kernels,
        max_nans_tolerated=max_nans_tolerated,
        band_to_cover=loci_separation_bins,
        cols_to_return=cols_to_return,
    )
    #
    _filter = partial(
        extract_filtered_pixels,
        enrichment_threshold=_enrichment_threshold,
        nnz_threshold=_nnz_threshold,
    )
    # ...
    # compose scoring and histogramming together
    _job = lambda tile: _filter(_score(tile))
    # standard multiprocessing implementation
    if nproc > 1:
        logging.info(f"creating a Pool of {nproc} workers to tackle {len(tiles)} tiles")
        pool = mp.Pool(nproc)
        map_ = pool.imap
        map_kwargs = dict(chunksize=int(np.ceil(len(tiles) / nproc)))
    else:
        logging.info("fallback to serial implementation.")
        map_ = map
        map_kwargs = {}
    try:
        scored_df_chunks = map_(_job, tiles, **map_kwargs)
    finally:
        if nproc > 1:
            pool.close()

    return scored_df_chunks

In [ ]:
# run pixels scoring and extraction/filtering on each tile ...
scored_df = score_pixels_OE_bulk(
    clr=clr,
    expected_indexed=exp_indexed,
    expected_value_col="balanced.avg",
    clr_weight_name="weight",
    tiles=tiles,
    kernels=k7,  # using k7 @10kb
    max_nans_tolerated=max_nans_tolerated,
    loci_separation_bins=bp_to_bins(max_loci_separation, binsize),
    nproc=12,
    cols_to_return=cols_to_return,
)
# stitch returned enriched pixels from each tile ...
enriched_pixels = pd.concat(scored_df, ignore_index=True)

# report ...
print(f"detected {len(enriched_pixels)} ...")

if False:
    # save enriched pixels for plotting/exploration and such ...
    enriched_pixels.to_csv(
        "enriched_pixels_10kb/5hr_2X_aug24.binpe",
        sep="\t",
        index=False,
    )

# Now we have a list of enriched pixels: `enriched_pixels`
 - bin coordinates only
 - with some extra info

# Density-cluster `enriched_pixels`
 - go OPTICS, consider DBSCAN ...
 - minimal cluster size is `5`
 - clustering radius is `33_000` bp ...

In [ ]:
# annotate enriched pixels with chrom, start, end info ...
enriched_pixels_annotated = cooler.annotate(
    enriched_pixels,
    clr.bins()[["chrom", "start", "end"]],
    replace=False
)
# Clustering is done independently for every region, therefore regions must be assigned:
enriched_pixels_annotated = assign_regions(enriched_pixels_annotated, hg38_arms)
# make regions categorical - such that sorting is preserved ...
enriched_pixels_annotated["region"] = pd.Categorical(enriched_pixels_annotated["region"], categories = included_arms)

# let's try density based clustering here ...
# we are working in cis strictly here - so just region is enough:
grp = enriched_pixels_annotated.groupby("region")

cols_rename = {
    ('chrom1', 'first'): "chrom1",
    ('start1', 'min'): "start1",
    ('end1', 'max'): "end1",
    ('chrom2', 'first'): "chrom2",
    ('start2', 'min'): "start2",
    ('end2', 'max'): "end2",
    ('bin1_id', 'min'): "bin1_id",
    ('bin1_id', 'ptp'): "bin1_width",
    ('bin2_id', 'min'): "bin2_id",
    ('bin2_id', 'ptp'): "bin2_width",
    ('count', 'sum'): "count",
    ('dist', 'min'): "dist",
    ('zl', 'sum'): "nnz",
    ('region', 'first'): "region",
    ('region', 'count'): "cluster_size",
}

_min_cluster_size = 5
_clustering_radius = 33_000  # in basepairs ...
_clustered_pix_region = []

for grp_region, df in grp:
    # cluster by region ...
    print(f"clustering {grp_region} ...")
    if len(df):
        _clust = OPTICS(
            max_eps=_clustering_radius,
            min_samples=_min_cluster_size
        ).fit( df[["start1","start2"]].to_numpy() )
        # assign labels back to the dataframe (per region) ...
        _cluster_labels = pd.Series(_clust.labels_).astype("str")
        df["labels"] = ( f"{grp_region}_" + _cluster_labels ).to_numpy()
        # filter out singletons ...
        df = df[ ~df["labels"].str.endswith("-1") ]
        # group by clustering labels ...
        grp = df.groupby(by="labels")
        chrdf = grp.agg({
            "chrom1" : "first",
            "start1" : "min",
            "end1" : "max",
            "chrom2" : "first",
            "start2" : "min",
            "end2" : "max",
            "bin1_id" : ["min", np.ptp],
            "bin2_id" : ["min", np.ptp],
            "count" : "sum",
            "dist" : "min",
            "zl" : "sum",
            "region" : ["first", "count"],
        })
        chrdf.columns = chrdf.columns.to_flat_index()
        chrdf = chrdf.rename(columns=cols_rename).reset_index()
        _clustered_pix_region.append(chrdf)
    else:
        print(f"WARNING: {grp_region} region is empty, skipping ...")


# concat regions together ...
clustered_pixels = pd.concat(_clustered_pix_region, ignore_index=True)
print(f"\nClustering yields {len(clustered_pixels)} pixel-clusters ...")

if False:
    clustered_pixels.to_csv(
        "clustered_pixels_10kb/5hr_2X_aug24.binpe",
        sep="\t",
        index=False,
    )

In [ ]:
clustered_pixels["dist"].hist(bins=100, log=False)
ax = plt.gca()
ax.set_xlabel("ID interactions genomic separation")

# Cluster, keeping individual pixels there [removing singletons]...
 - an attemp to improve accuracy of anchor calling ...
 - i.e. make only "cloud-forming" pixels participate in the anchor formation
 - remove singletons altogether ...

In [ ]:
_clustered_pixels_all = []

# let's try density based clustering here ...
# we are working in cis strictly here - so just region is enough:
grp = enriched_pixels_annotated.groupby("region", observed=True)
for grp_region, df in grp:
    # cluster by region ...
    if len(df):
        print(f"clustering {len(df)} pixels from {grp_region} ...")
        _clust = OPTICS(
            max_eps=_clustering_radius,
            min_samples=_min_cluster_size
        ).fit(
            df.eval(
                """
                x = (start1 + end1)*0.5
                y = (start2 + end2)*0.5
                """
            )[["x","y"]].to_numpy()
        )
        # assign labels back to the dataframe (per region) ...
        _cluster_labels = pd.Series(_clust.labels_).astype("str")
        df["labels"] = ( f"{grp_region}_" + _cluster_labels ).to_numpy()
        # filter out singletons ...
        df = df[ ~df["labels"].str.endswith("-1") ]
        _clustered_pixels_all.append(df)
    else:
        print(f"WARNING: {grp_region} region is empty, skipping ...")

# # concat and assign ..
# enriched_pixels_annotated["labels"] = pd.concat(_labels).to_numpy()
clustered_pixels_all = pd.concat(_clustered_pixels_all, ignore_index=True)
print(f"\nClustering yields {len(clustered_pixels_all)} pixel-clusters ...")

# Save `enriched_pixels` and `clustered_pixels` on disk ...

In [ ]:
# ...
dirname = "enriched_pixels_10kb"
fname = f"{dirname}/5hr_2X_second.binpe"
print(f"saving {len(enriched_pixels)} enriched pixels to {fname} in BEDPE format (only bin_ids actually @10kb) ...")
enriched_pixels.to_csv(fname, sep="\t", index=False)
# ...
# w/o singletons
fname = f"{dirname}/5hr_2X_second.no_singletons.binpe"
print(f"saving {len(clustered_pixels_all)} enriched pixels (w/o singletons) to {fname} in BEDPE format (only bin_ids actually @10kb) ...")
clustered_pixels_all.to_csv(fname, sep="\t", index=False)

# ...
dirname = "native_comps_10kb"
fname = f"{dirname}/5hr_2X_second.bedpe"
print(f"saving {len(clustered_pixels)} clustered pixels to {fname} in BEDPE format ...")
clustered_pixels.to_csv(fname, sep="\t", index=False)

# Enriched pixels coverage "games" to define anchors ...
 - define coverage of enriched pixels (and enriched ones w/o singletons)
 - check out its distributions and example tracks (save it is bigwig as well) ...
 - use `scipy.signal.find_peaks` to detect anchors and their footprint
 - `_val_thresh = 7` - is the bottom floor for the coverage of enriched pixels ...


In [ ]:
def get_bin_coverage(df):
    """
    df with bin1_id and bin2_id columns
    coverage for every bin that is out there ...
    """
    # simply count "valencies" of enriched pixels i-s and j-s - and sum tham up togeher ...
    b1cov = df.groupby("bin1_id").size()
    b2cov = df.groupby("bin2_id").size()
    b1cov.index.name = "bin"
    b2cov.index.name = "bin"
    return b1cov.add(b2cov, fill_value=0)

def calculate_valencies(
    bed_df,  # must be output of bedpe_to_anchors, which in turn is a clustering inside
    bedpe_df,
    cluster_colname = "cluster",
    valency_colname = "valency",
    bed_cols = ["chrom", "cluster_start", "cluster_end"],
    bedpe_cols1 = ["chrom1", "start1", "end1"],
    bedpe_cols2 = ["chrom2", "start2", "end2"],
):
    """
    calculate valencies of a given anchors, given the bedpe ...
    """
    if cluster_colname not in bed_df.columns:
        raise ValueError("bed_df does not seem to be the result of bedpe_to_anchors/clustering ...")
    # overlap combined anchors with the left anchors to see how many "dots" we overlap ...
    anchors_left = bioframe.overlap(
        bed_df,
        bedpe_df,
        how='left',
        cols1=bed_cols,
        cols2=bedpe_cols1,
    ).dropna( subset=[f"{c}_" for c in bedpe_cols1] )
    # overlap combined anchors with the right anchors to see how many "dots" we overlap ...
    anchors_right = bioframe.overlap(
        bed_df,
        bedpe_df,
        how='left',
        cols1=bed_cols,
        cols2=bedpe_cols2,
    ).dropna( subset=[f"{c}_" for c in bedpe_cols2] )
    _num_clusters = len(bed_df)
    # sanity check here ... - make sure we cover all of the cluster that are available ...
    assert ( bed_df[cluster_colname].sort_values() == np.arange(_num_clusters) ).all()
    # ...
    _empty_clust_series = pd.Series(
        data=np.zeros(_num_clusters),
        index=pd.Index(data=np.arange(_num_clusters), name=cluster_colname),
        name="count"
    )
    # calculate valencies ...
    _valencies = (_empty_clust_series + anchors_left[cluster_colname].value_counts()).fillna(0) \
                + (_empty_clust_series + anchors_right[cluster_colname].value_counts()).fillna(0)
    # assign valencies back to anchors bed_df - carefully !
    # _valencies are indexed using cluster_id - s
    bed_df_clust_indexed = bed_df.set_index(cluster_colname)
    bed_df_clust_indexed[valency_colname] = _valencies.astype(int)
    #
    return bed_df_clust_indexed.reset_index()


def coverage_to_anchors(cov_df, full_bintable, coverage_threshold, clustered_pixels=None):
    """
    """
    # _idx = rich_pix_vals[rich_pix_vals > _val_threshold].index
    _rich_bins = full_bintable.loc[cov_df[cov_df > coverage_threshold].index]

    _new_anchors = bioframe.merge(_rich_bins)
    _new_anchors = _new_anchors.reset_index().rename(columns={"index":"cluster"})
    # add few extra columns and save it ...
    _new_anchors["size"] = _new_anchors.eval("end - start")
    if clustered_pixels is not None:
        # calculate valency the old way by overlapping with actually clustered pixels ...
        _new_anchors = calculate_valencies(
            _new_anchors,  # must be output of bedpe_to_anchors, which in turn is a clustering inside
            clustered_pixels,
            cluster_colname = "cluster",
            valency_colname = "valency",
            bed_cols = ["chrom", "start", "end"],
            bedpe_cols1 = ["chrom1", "start1", "end1"],
            bedpe_cols2 = ["chrom2", "start2", "end2"],
        )
    else:
        _new_anchors["valency"] = 10
    # return ...
    return _new_anchors


# calculate pixels coverage ...
rich_pix_vals = get_bin_coverage(enriched_pixels)
rich_clust_pix_vals = get_bin_coverage(clustered_pixels_all)

# take empty bins and fill non-zero coverage with rich_clust_pix_vals ...
# use value threshold to define the "floor" of the pixel coverage ...
_val_thresh = 7

_bins = clr.bins()[:]
_bins.index.name = "bin"
_bins["cov"] = rich_clust_pix_vals
_bins["cov"] = _bins["cov"].fillna(0)
_arr = _bins["cov"].to_numpy()
# detect praks on the coverage track ...
_peaks, _props = find_peaks(
    np.clip(_arr, _val_thresh, None),
    prominence=(None,None),
    distance=5,
)
print(f"{len(_peaks)} peaks were called ...")
# extract left/right boundaries of every peak ...
_lefts = _props["left_bases"]
_rights = _props["right_bases"]
_arr_clipped = np.clip(_arr, _val_thresh, None)


### save coverage track as bigWig ...

In [ ]:
bw_fname = "pix_clust_cov.bw"
# save that coverage track as bigwig ...
import pybigtools
_bins_tobw = _bins.copy()
_bins_tobw["chrom"] = _bins_tobw["chrom"].astype("str")
_bins_tobw = _bins_tobw[["chrom", "start", "end", "cov"]]
bw = pybigtools.open(bw_fname, mode="w")
bw.write(
    hg38_chromsizes.to_dict(),
    iter(tuple(t) for t in _bins_tobw.sort_values(by=["chrom","start"]).itertuples(index=False)),
)

## Visualize the anchor detection procdeure ...

In [ ]:
fig = plt.figure(figsize=(10,10), layout="tight")
gs = GridSpec(3, 2, figure=fig)
ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1])
ax3 = fig.add_subplot(gs[1,:])
ax4 = fig.add_subplot(gs[2,:])
fig.suptitle("enriched pixels coverage exploration, aka 'valency'")

_max_val = 200
ax1.hist(rich_pix_vals.to_numpy(), bins=np.linspace(0,_max_val,_max_val+1), log=True);
ax1.hist(rich_clust_pix_vals.to_numpy(), bins=np.linspace(0,_max_val,_max_val+1), log=True, alpha=0.88);
ax1.axvline(_val_thresh,color="red")
ax1.set_xlabel("valency")
ax1.set_ylabel("hist. counts")
ax1.set_xlim(0, _max_val)
_max_val = 30
ax2.hist(rich_pix_vals.to_numpy(), bins=np.linspace(0,_max_val,_max_val+1), log=True);
ax2.hist(rich_clust_pix_vals.to_numpy(), bins=np.linspace(0,_max_val,_max_val+1), log=True, alpha=0.88);
ax2.axvline(_val_thresh,color="red")
ax2.set_xlabel("valency")
ax2.set_title("hist zoom in")
ax2.set_xlim(0, _max_val)

# example of coverage track and stuff within a region ...
_chrom = "chr6"
_reg_start = 127_000_000
_reg_length = 10_000_000
_from, _to = clr.extent((_chrom, _reg_start, _reg_start+_reg_length))
# print(_from, _to)
rich_pix_vals_plot = pd.Series(index=np.arange(_from,_to)).add(rich_pix_vals.loc[_from: _to], fill_value=0)
rich_clust_pix_vals_plot = pd.Series(index=np.arange(_from,_to)).add(rich_clust_pix_vals.loc[_from: _to], fill_value=0)

ax3.plot(rich_pix_vals_plot, marker=".", lw=0.5, label="all enriched pixels")
ax3.plot(rich_clust_pix_vals_plot, marker=".", lw=0.5, label="all enriched pixels (w/o singletons)")
ax3.axhline(_val_thresh, color="red")
ax3.set_xlim(_from, _to)
ax3.legend(frameon=False)
ax3.set_title(f"example coverage: {_chrom}:{_reg_start}-{_reg_start+_reg_length}")

ax4.plot(_arr_clipped[_from: _to])
ax4.plot(_peaks - _from, _arr_clipped[_peaks], "x" )
ax4.plot(_lefts - _from, _arr_clipped[_lefts], "o" )
ax4.plot(_rights - _from, _arr_clipped[_rights], "o" )
ax4.set_xlim(0, _to-_from )
ax4.set_title(f"clipped coverage with detected anchors/peaks ...")
# ax.axhline(_val_thresh, color="red")


# try adding an axes manually ...
fig.add_axes([0.88,0.01,0.1,0.02])

## Anchors from peaks (clipped coverage way) ...

In [ ]:
_new_new_anchors = coverage_to_anchors(
    pd.Series(data=10., index=_peaks),
    clr.bins()[:],
    0,
    clustered_pixels=clustered_pixels
)
# modify their size to the actual footprint of the peak ...
# _new_new_anchors["size"] = _props["right_bases"] - _props["left_bases"]
_new_new_anchors["peak_start"] = clr.bins()[:].loc[_lefts, "start"].to_numpy()
_new_new_anchors["peak_end"] = clr.bins()[:].loc[_rights, "end"].to_numpy()
# ...
_new_new_anchors["size"] = _new_new_anchors.eval("peak_end - peak_start")

In [ ]:
_new_new_anchors["size"].hist(bins=30)
ax = plt.gca()
ax.set_xlabel("ID anchor footprint, bp")
_new_new_anchors

In [ ]:
# save it ...
_cols_save = ["chrom", "start", "end", "cluster", "size", "valency", "peak_start", "peak_end", "size"]
dirname = "ID_anchors"
fname = f"{dirname}/5hr_2X_enrichment.pixel_derived.signal_peaks.bed"
_new_new_anchors[_cols_save].to_csv(
    fname,
    index=False,
    sep="\t",
)


## Other anchors ...

In [ ]:
_new_anchors = coverage_to_anchors(
    rich_pix_vals,
    clr.bins()[:],
    _val_thresh,
    clustered_pixels=clustered_pixels
)

# save it ...
_cols_save = ["chrom","start","end","cluster","size","valency"]
dirname = "ID_anchors"
fname = f"{dirname}/5hr_2X_enrichment.pixel_derived.bed"
_new_anchors[_cols_save].to_csv(
    fname,
    index=False,
    sep="\t",
)

_new_anchors_nosing = coverage_to_anchors(
    rich_clust_pix_vals,
    clr.bins()[:],
    _val_thresh,
    clustered_pixels=clustered_pixels
)

# save it ...
_cols_save = ["chrom","start","end","cluster","size","valency"]
dirname = "ID_anchors"
fname = f"{dirname}/5hr_2X_enrichment.pixel_derived.no_singletons.bed"
_new_anchors_nosing[_cols_save].to_csv(
    fname,
    index=False,
    sep="\t",
)


# Now define some plotting functions and draw enriched pixels and clusters on heatmaps ...

In [ ]:
def rectangles_around_dots(dots_bins_df, the_tile, loc="upper", lw=1, ec="cyan", fc="none"):
    rectangle_kwargs = dict(lw=lw, ec=ec, fc=fc)
    # parse the tile
    _, tspan1, tspan2 = the_tile
    # select only visible pixels :
    _the_dots = dots_bins_df \
        .query("""(@tspan1[0] < bin1_id < @tspan1[1]) & \
                  (@tspan2[0] < bin2_id < @tspan2[1]) """) \
        .eval("""b1 = bin1_id - @tspan1[0]
                 b2 = bin2_id - @tspan2[0] """)
    print(f"{len(_the_dots)} pixels are visible out of {len(dots_bins_df)} ...")
    # iterate over visible pixels...
    for b1, b2 in _the_dots[["b1", "b2"]].itertuples(index=False):
        w1 = w2 = 1
        if loc == "upper":
            yield patches.Rectangle((b2, b1), w2, w1, **rectangle_kwargs)
        elif loc == "lower":
            yield patches.Rectangle((b1, b2), w1, w2, **rectangle_kwargs)
        else:
            raise ValueError("loc has to be uppper or lower")

# in a specific region, and exposing importnat plotting parameters
def rectangles_around_dots_ww(dots_bins_df, the_tile, loc="upper", lw=1, ec="cyan", fc="none", halo=30_000):
    rectangle_kwargs = dict(lw=lw, ec=ec, fc=fc)
    # parse the tile
    _, tspan1, tspan2 = the_tile
    # select only visible "boxes" :
    _the_dots = dots_bins_df \
        .query("""(@tspan1[0] - @halo < bin1_id < @tspan1[1] + @halo) & \
                  (@tspan2[0] - @halo < bin2_id < @tspan2[1] + @halo) """) \
        .eval("""b1 = bin1_id - @tspan1[0]
                 b2 = bin2_id - @tspan2[0] """)
    print(f"{len(_the_dots)} pixels are visible out of {len(dots_bins_df)} ...")
    for b1, b2, w1, w2 in _the_dots[["b1", "b2", "bin1_width", "bin2_width"]].itertuples(index=False):
        if loc == "upper":
            yield patches.Rectangle((b2, b1), w2+1, w1+1, **rectangle_kwargs)
        elif loc == "lower":
            yield patches.Rectangle((b1, b2), w1+1, w2+1, **rectangle_kwargs)
        else:
            raise ValueError("loc has to be uppper or lower")

In [ ]:
# display that region of interest as a heatmap ...
i_slider = widgets.IntSlider(min=0, max=100, step=1, value=24, description="tile id :")

@interact(i=i_slider)
def f(i):
    # region_name, tile_span_i, tile_span_j = 'chr1_p', (2995, 3505), (2995, 3505)
    _tile_id = i
    _the_tile = tiles[_tile_id]
    # for _tile_id, _the_tile in enumerate(tiles[69:], start=69):
    region_name, tile_span_i, tile_span_j = _the_tile
    tile_start_ij = (tile_span_i[0], tile_span_j[0])
    lazy_exp = LazyToeplitz(
        exp_indexed.loc[region_name, region_name]["balanced.avg"].to_numpy()
    )
    # RAW observed matrix slice:
    observed = clr.matrix()[slice(*tile_span_i), slice(*tile_span_j)]
    expected = lazy_exp[slice(*tile_span_i), slice(*tile_span_j)]

    # let's figure out slices' coordinates ....
    _bins_i = clr.bins()[slice(*tile_span_i)]
    _bins_j = clr.bins()[slice(*tile_span_j)]
    _chrom_i, _start_i, _end_i = _bins_i.iloc[0]["chrom"], _bins_i.iloc[0]["start"], _bins_i.iloc[-1]["end"]
    _chrom_j, _start_j, _end_j = _bins_j.iloc[0]["chrom"], _bins_j.iloc[0]["start"], _bins_j.iloc[-1]["end"]


    f, (axleft, axright) = plt.subplots(nrows=1,ncols=2,figsize=(24,11),sharex=True,sharey=True)
    f.suptitle(f"tile # {_tile_id} {(_chrom_i, _start_i, _end_i)} {(_chrom_j, _start_j, _end_j)}",y=0.9)

    print(f"tile # {_tile_id} {(_chrom_i, _start_i, _end_i)} {(_chrom_j, _start_j, _end_j)}")

    axleft.imshow(
        observed,
        cmap="fall",
        interpolation="none",
        norm=LogNorm(0.0001, 0.01)
    )
    axright.imshow(
        (observed/expected),
        cmap="RdBu_r",
        interpolation="none",
        norm=LogNorm(0.25, 4)
    )

    # draw rectangular "boxes" around enriched pixels ...
    for box in rectangles_around_dots(
        enriched_pixels,
        _the_tile,
        loc="upper",
        lw=1,
        ec="cyan",
        fc="none"
    ):
        axleft.add_patch(box)

    # draw rectangular "boxes" around clustered pixels ...
    for box in rectangles_around_dots_ww(
        clustered_pixels,
        _the_tile,
        loc="upper",
        lw=1,
        ec="k",
        fc="none",
        halo=100,
    ):
        axleft.add_patch(box)
    for box in rectangles_around_dots_ww(
        clustered_pixels,
        _the_tile,
        loc="upper",
        lw=1,
        ec="darkgreen",
        fc="none",
        halo=100
    ):
        axright.add_patch(box)


# Below is a sanity Pileup exploration of the `clustered_pixels`
 - all
 - by distance
 - by distance and chromosome/arm
 - by cluster size ...

### Goal is to demonstrate that the "calls" are not spurious and look "tight" individually as well ...

In [ ]:
_flank = 100_000 # Length of flank to one side from the boundary, in basepairs
# create the stack of snips:
fullstack = cooltools.pileup(
    clr,
    clustered_pixels,
    view_df=hg38_arms,
    expected_df=exp,
    flank=_flank,
    nproc=12
)

### test the pileup theory here ...

are snippets returned in the same order they are in the dataframe ...

In [ ]:
# just a single pileup to see ...
img = plt.imshow(
    np.nanmean(fullstack, axis=0),
    norm=LogNorm(vmin=1/4,vmax=4),
    interpolation="none",
    extent=[-_flank//1000, _flank//1000, -_flank//1000, _flank//1000],
    cmap='RdBu_r'
)
plt.colorbar(img)

In [ ]:
n_examples = len(clustered_pixels)

@interact(i=(0, n_examples-1))
def f(i):
    _snip = fullstack[i, :, :]
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3,figsize=[15,5])
    # ax1 ...
    ax1.plot(_snip.sum(axis=0))
    ax1.set_xlim(0,2*_flank//binsize)
    ax1.set_xticks([0,_flank//binsize, 2*_flank//binsize])
    ax1.set_xticklabels([-_flank//1000,0,_flank//1000])
    # ax2 ...
    ax2.plot(_snip.sum(axis=1))
    ax2.set_xlim(0,2*_flank//binsize)
    ax2.set_xticks([0,_flank//binsize, 2*_flank//binsize])
    ax2.set_xticklabels([-_flank//1000,0,_flank//1000])
    # ax3 ...
    ax3.imshow(
        _snip,
        norm=LogNorm(vmin=1/4,vmax=4),
        interpolation="none",
        extent=[-_flank//1000, _flank//1000, -_flank//1000, _flank//1000],
        cmap='coolwarm'
    )
    ax3.axvline(0, c='g', ls=':')
    ax3.axhline(0, c='g', ls=':')


In [ ]:
# f, axs = plt.subplots(nrows=1,ncols=len(stacks)+1,figsize=(20,5),width_ratios=[1]*len(stacks)+[0.05])
nquant = 7
dist_bins = np.linspace(0,1,nquant+1)
f, axs = plt.subplots(
    nrows=1,
    ncols=nquant+1,
    figsize=(3.2*nquant,3),
    width_ratios=[1]*nquant+[0.05]
)
snp_grp = clustered_pixels.groupby(
    pd.qcut(clustered_pixels["dist"], dist_bins)
)

for i, (ax, (_q, _mtx)) in enumerate(zip(axs, snp_grp.groups.items())):
    _ccc = ax.imshow(
        np.nanmean(fullstack[_mtx,:,:], axis=0),
        cmap='RdBu_r',
        norm=LogNorm(vmin=1/4,vmax=4)
    )
    ticks_pixels = np.linspace(0, _flank*2//binsize, 5)
    ticks_kbp = ((ticks_pixels-ticks_pixels[-1]/2)*binsize//1000).astype(int)
    ax.set_title(f"{_q}")
    ax.set_xticks(ticks_pixels, ticks_kbp)
    ax.set_yticks(ticks_pixels, ticks_kbp)
    ax.set_xlabel('relative position, kbp')
    if i<1:
        ax.set_ylabel('relative position, kbp')
    if i==nquant-1:
        plt.colorbar(_ccc, label = 'obs/exp', cax=axs[i+1])


## try by pileup by region and by distance at the same time ! ...

In [ ]:
reg_grp = clustered_pixels.groupby( "region" )

nquant = 7
dist_bins = np.linspace(0,1,nquant+1)

for reg, gdf in reg_grp:
    if not len(gdf):
        print(f"... skipping {reg}, cause empty ...")
        continue
    f, axs = plt.subplots(
        nrows=1,
        ncols=nquant+1,
        figsize=(3.2*nquant,3),
        width_ratios=[1]*nquant+[0.05]
    )
    snp_grp = gdf.groupby( pd.qcut(gdf["dist"], dist_bins) )
    f.suptitle(f"{reg}")

    for i, (ax, (_q, _mdf)) in enumerate(zip(axs, snp_grp)):
        _ccc = ax.imshow(
            np.nanmean(fullstack[_mdf.index,:,:], axis=0),
            cmap='RdBu_r',
            norm=LogNorm(vmin=1/4,vmax=4)
        )
        ticks_pixels = np.linspace(0, _flank*2//binsize, 5)
        ticks_kbp = ((ticks_pixels-ticks_pixels[-1]/2)*binsize//1000).astype(int)
        ax.set_title(f"{_q}: {len(_mdf)}")
        ax.set_xticks(ticks_pixels, ticks_kbp)
        ax.set_yticks(ticks_pixels, ticks_kbp)
        ax.set_xlabel('relative position, kbp')
        if i<1:
            ax.set_ylabel('relative position, kbp')
        if i==nquant-1:
            plt.colorbar(_ccc, label = 'obs/exp', cax=axs[i+1])

### try a few more groupings ...

In [ ]:
gfeat_name = "cluster_size"
the_grp = clustered_pixels.groupby( gfeat_name )

nquant = 5
dist_bins = np.linspace(0,1,nquant+1)



for _gname, gdf in the_grp:
    if len(gdf)>20:
        # split by distnace - cause enough data ...
        f, axs = plt.subplots(
            nrows=1,
            ncols=nquant+1,
            figsize=(3.2*nquant,3),
            width_ratios=[1]*nquant+[0.05]
        )
        snp_grp = gdf.groupby( pd.qcut(gdf["dist"], dist_bins) )
        f.suptitle(f"{gfeat_name}: {_gname}")

        for i, (ax, (_q, _mdf)) in enumerate(zip(axs, snp_grp)):
            _ccc = ax.imshow(
                np.nanmean(fullstack[_mdf.index,:,:], axis=0),
                cmap='RdBu_r',
                norm=LogNorm(vmin=1/4,vmax=4)
            )
            ticks_pixels = np.linspace(0, _flank*2//binsize, 5)
            ticks_kbp = ((ticks_pixels-ticks_pixels[-1]/2)*binsize//1000).astype(int)
            ax.set_title(f"{_q}: {len(_mdf)}")
            ax.set_xticks(ticks_pixels, ticks_kbp)
            ax.set_yticks(ticks_pixels, ticks_kbp)
            ax.set_xlabel('relative position, kbp')
            if i<1:
                ax.set_ylabel('relative position, kbp')
            if i==nquant-1:
                plt.colorbar(_ccc, label = 'obs/exp', cax=axs[i+1])
    else:
        print(f"cannot split {_gname} by dist, cause too small ...")
        f, ax = plt.subplots(
            nrows=1,
            ncols=1,
            figsize=(5,5),
        )
        f.suptitle(f"{gfeat_name}: {_gname}")
        _ccc = ax.imshow(
            np.nanmean(fullstack[gdf.index,:,:], axis=0),
            cmap='RdBu_r',
            norm=LogNorm(vmin=1/4,vmax=4)
        )
        ticks_pixels = np.linspace(0, _flank*2//binsize, 5)
        ticks_kbp = ((ticks_pixels-ticks_pixels[-1]/2)*binsize//1000).astype(int)
        ax.set_title(f"{_q}: {len(_mdf)}")
        ax.set_xticks(ticks_pixels, ticks_kbp)
        ax.set_yticks(ticks_pixels, ticks_kbp)
        ax.set_xlabel('relative position, kbp')
        ax.set_ylabel('relative position, kbp')
        plt.colorbar(_ccc, label = 'obs/exp')